In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Lambda
from tensorflow.keras.backend import expand_dims, repeat_elements
import numpy as np
import h5py

In [ ]:
print('GPU name: ', tf.config.experimental.list_physical_devices('GPU'))

In [7]:
INPUT_STEPS = 32
OUTPUT_STEPS = 16

In [3]:
class DataGenerator(Sequence):
    """Generates data for Keras"""

    def __init__(self, paths, batch_size=16, dim=(64, 64),
                 shuffle=True, input_steps=16, output_steps=8):
        """Initialization"""
        self.dim = dim
        self.file_batch_size = 2048
        self.batch_size = batch_size
        self.n_batches = self.file_batch_size // self.batch_size
        self.file_paths = paths
        self.shuffle = shuffle
        self.current_file_loaded = (None, None)
        self.steps = (input_steps, output_steps)
        self.indexes = np.arange(len(self.file_paths) * int(self.file_batch_size / self.batch_size))
        self.on_epoch_end()

    def load_file(self, i):
        with h5py.File(self.file_paths[i], "r") as f:
            self.current_file_loaded = i, (f["data/X"][:], f["data/y"][:])

    def __len__(self):
        """Denotes the number of batches per epoch"""
        return len(self.file_paths) * self.n_batches
        # TODO: Arreglar esto

    def __getitem__(self, index):
        """Generate one batch of data"""
        # Generate indexes of the batch
        file_index = index // self.n_batches
        current_file_index, _ = self.current_file_loaded
        if file_index != current_file_index:
            self.load_file(file_index)
        _, (X, y) = self.current_file_loaded
        index_inside_file = index % self.n_batches
        i = index_inside_file * self.batch_size
        inp, out = self.steps
        X = np.log(X[i:(i + self.batch_size), -inp:] + 1.0, dtype=np.float32)
        y = y[i:(i + self.batch_size), :out].astype(np.float32)
        y[y > 1.0] = np.float32(1.0)
        return X, y

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        self.indexes = np.arange(len(self.file_paths))
        if self.shuffle:
            np.random.shuffle(self.indexes)
            np.random.shuffle(self.file_paths)
            self.current_file_loaded = (None, None)

In [11]:
from tensorflow.keras.losses import binary_crossentropy, BinaryCrossentropy
from tensorflow.keras.optimizers import Adam

def get_compiled_model():

    inp = layers.Input(shape=(INPUT_STEPS, 64, 64, 1))

    # Probar normalizar dividiendo entre 300
    # Probar normalizar log(x+1)
    
    x = layers.ConvLSTM2D(
        filters=48,
        kernel_size=(5, 5),
        #padding="same",
        return_sequences=True,
        activation="relu",
    )(inp)
    x = layers.ConvLSTM2D(
        filters=64,
        kernel_size=(3, 3),
        #padding="same",
        activation="relu",
    )(x)
    x = Lambda(lambda x: repeat_elements(expand_dims(x, axis=1), OUTPUT_STEPS, 1))(x)
    x = layers.ConvLSTM2D(
        filters=64,
        kernel_size=(1, 1),
        #padding="same",
        return_sequences=True,
        activation="relu",
    )(x)
    x = layers.ConvLSTM2D(
        filters=64,
        kernel_size=(1, 1),
        #padding="same",
        return_sequences=True,
        activation="relu",
    )(x)
    x = layers.Conv2D(
        filters=64, kernel_size=(3, 3), activation="sigmoid", padding="same"
    )(x)


    model = Model(inp, x)
    model.compile(
        loss=BinaryCrossentropy(), 
        optimizer=Adam(),
    )
    return model


In [12]:
import os
folder = "../data/exp_pro/GLM-L2-LCFA_8km_5m_boxes/2019/BATCH_SIZE=2048_CLUSTERING_MIN_CLUSTER_SIZE=512_CLUSTERING_RADIUS=24_CLUSTERING_THRESHOLD=0.0_CLUSTERING_TIME_FACTOR=8_HEIGHT=64_RNG_SEED=42_SLIDING_WINDOW_STEPS=4_TIME_DELTA_AFTER=4_TIME_DELTA_BEFORE=6_TIME_IN=32_TIME_OUT=16_WIDTH=64/h5/"
files = np.array([os.path.join(folder, x) for x in os.listdir(folder)][:-1])

train_val_test_split = (.7, .2, .1)

def split(arr, splits):
    arr = np.array(arr)
    idxs = np.arange(len(arr))
    np.random.shuffle(idxs)
    n_splits = list(int(len(arr)*x) for x in splits)
    datasets = []
    start = 0
    n_splits[-1] += len(arr) - sum(n_splits)
    n_splits = tuple(n_splits)
    for split in n_splits:
        datasets.append(arr[idxs[start:start+split]])
        start += split
    return tuple(datasets)

train_files, val_files, test_files = split(files, train_val_test_split)

train_generator = DataGenerator(train_files, input_steps=INPUT_STEPS, output_steps=OUTPUT_STEPS)
val_generator = DataGenerator(val_files, input_steps=INPUT_STEPS, output_steps=OUTPUT_STEPS)
test_generator = DataGenerator(test_files, input_steps=INPUT_STEPS, output_steps=OUTPUT_STEPS)

In [13]:
# [0, 1>
# [1, 5>
# [5, ...>
# [..., 80>
# [80, inf>

In [15]:
model = get_compiled_model()

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TerminateOnNaN


filepath = "saved-model-{epoch:02d}-{val_loss:.6f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, mode='auto', save_weights_only=False)
early_stopping = EarlyStopping(monitor="val_loss", patience=10)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", patience=5)
terminate_nan = TerminateOnNaN()
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 64, 64, 1)]   0         
                                                                 
 conv_lstm2d_4 (ConvLSTM2D)  (None, 32, 60, 60, 48)    235392    
                                                                 
 conv_lstm2d_5 (ConvLSTM2D)  (None, 58, 58, 64)        258304    
                                                                 
 lambda_1 (Lambda)           (None, 16, 58, 58, 64)    0         
                                                                 
 conv_lstm2d_6 (ConvLSTM2D)  (None, 16, 58, 58, 64)    33024     
                                                                 
 conv_lstm2d_7 (ConvLSTM2D)  (None, 16, 58, 58, 64)    33024     
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 58, 58, 64)    3692

In [ ]:
epochs = 5

model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator,
    callbacks=[
        early_stopping,
        reduce_lr,
        checkpoint,
        terminate_nan,
    ],
)

In [ ]:
model.evaluate(test_generator)